In [1]:
import numpy as np


def read_data(day, kind):
    path = f'../data/{kind}-day{day}.txt'
    with open(path) as f:
        data = f.read().splitlines()
    return data

# Day 4

In [338]:


data = read_data(4, 'data')
test = read_data(4, 'test')

def get_card_input(card_list):  
    '''given list of 5 num str returns subdivided cards  '''
    n = len(card_list)
    card_input = []
    for card in np.arange(0, n, 5):
        card_input.append(card_list[card:card+5])
    return card_input 

def create_input(data):
    calls = data[0].split(',')
    card_list = [x.strip().split() for x in data[1:] if x != '']
    card_input = get_card_input(card_list)
    return card_input, calls

    
cards_in, calls_in = create_input(data)
test_cards_in, test_calls_in = create_input(test)


In [611]:
#call a number 

#play bingo

#id the winner

#class to track a card

class Card:
    def __init__(self, card_input):
        self.input = card_input
        self.calls = []
        self.valid_calls = []
        self.marked_locations = []
        self.win = False
        self.process_card()
        
    def process_card(self):
        '''numbers each space 1-25, dict includes val and 
        marked/un'''
        nums = sum(self.input, [])
        self.num_list = nums
        self.card_dict = dict(enumerate(nums, 1))
        self.num_dict = {v:k for k, v in self.card_dict.items()}
        self.marked_dict = {k:0 for k, v in self.card_dict.items()}
    
    def mark_space(self, numStr):
        '''marks space as being called/not, 
        first checks a dict of numbers keys 
        for its its included at all
        
        second look up its spot
        
        third go to the ground truth dict and update it
        
        '''
        self.calls.append(numStr)
        try:
            location = self.num_dict[numStr]
            self.valid_calls.append(numStr)
            self.marked_locations.append(location)
        except:
            return
            
    
    def check_for_marks(self, check_type):
        '''checks rows for a win; 
        
        Look through relevant numbers in the dict; 
        see if its marked.
        '''
        check_dict = {'rows': {'start_cols' : np.arange(1, 26, 5),
                               'overlap_cols' : lambda num: np.arange(num, 
                                                          num+5, 1)
                               
                              },
                      'cols': {'start_cols' : np.arange(1, 6, 1),
                               'overlap_cols' : lambda num: np.arange(num, 
                                                          26, 5)
                               
                              }
                     }
        
        start = check_dict[check_type]['start_cols']
        overlap = check_dict[check_type]['overlap_cols']
        
        
        remainders = [x for x in start
                      if x in self.marked_locations]
        
        if remainders == []:
            return
        
        
        for num in remainders:
            overlap_list = [x for x in self.marked_locations
                       if x in overlap(num)]

            if len(overlap_list) == 5:
                return True
        return False
    
    def check_for_win(self):
        '''checks rows and columns after marking_space
        and returns whether card has won
        '''
        if len(self.marked_locations) < 5:
            return False
        
        if self.check_for_marks('cols'):
            return True
        elif self.check_for_marks('rows'):
            return True
        else:
            return False
        
    def get_score(self, numStr):
        unmarked = [int(x) for x in self.num_list 
            if x not in self.valid_calls]
#         print(unmarked)
#         print(numStr)
        self.score = sum(unmarked) * int(numStr) 
        return

    def get_call(self, numStr):
        if self.win:
            return
        else:
            self.mark_space(numStr)
            self.win = self.check_for_win()
            self.get_score(numStr)
            return self.win


In [608]:
def play_bingo(cards_list, calls, stop=1):

    #set up all the cards, iter through list to create dict of cards
    cards = []
    win_cards = []
    for card_nums in cards_list:
        cards.append(Card(card_nums))
    
    #call number, expects list of strs
    for c in calls:
        for x in cards:
            if x.get_call(c):
                win_cards.append((x, x.score))
                if len(win_cards) == stop:
                    lose = [x for x in cards 
                            if x not in 
                            [y[0] for y in win_cards]]
                    return win_cards, lose
                


In [612]:
win, lose = play_bingo(test_cards_in, test_calls_in)
win[-1][1]

4512

In [613]:
win, lose = play_bingo(cards_in, calls_in)
win[-1][1]

55770

In [609]:
n = len(test_cards_in)
win, lose = play_bingo(test_cards_in, test_calls_in, n)
win[-1][1]

1924

In [610]:
n = len(cards_in)
win, lose = play_bingo(cards_in, calls_in, n)
win[-1][1]

2980

# Day 5

In [2]:
test = read_data(5, 'test')
data = read_data(5, 'data')

In [3]:
def get_pairs(data):
    '''returns 2 by2 np array for each xy pairs'''
    s_data = [[x.split(',') for x in (x.split(' -> '))] for x in data]
    arr = np.array(s_data, dtype=int)
    return [(x[0], x[1]) if x[0][0] < x[1][0] else (x[1], x[0]) for x in arr]

input_data = get_pairs(test)

In [4]:
#[(x[0], x[1]) if x[0][0] < x[1][0] else (x[1], x[0]) for x in input_data]

In [5]:
def remove_diags(input_data):
    return [x for x in input_data if (x[0][0] == x[1][0]) | (x[0][1] == x[1][1])]

In [40]:
def get_interpolations(pairs):
    '''given array of 2 pairs of coords, 
    returns a list of tuples with all interpolated points
    '''
    xp = [x[0] for x in pairs]
    yp = [x[1] for x in pairs]

    if np.diff(xp) == 0:
        yp = sorted(yp)
        interp_for = np.arange(yp[0], yp[1]+1, 1, float)
        return list(zip(np.ones(len(interp_for))*xp[0], interp_for))
        
    interp_for = np.arange(xp[0], xp[1]+1, 1, dtype=float)
    
    out = np.interp(interp_for, xp, yp)
    
    return list(zip(interp_for, out))

def get_all_points(input_data):
    #points_list = []
    p_dict = {}
    for pair in input_data:
        #points_list= (get_interpolations(pair))
        #print(len(points_list))
        for point in get_interpolations(pair):
            try:
                p_dict[point] += 1
            except:
                p_dict[point] = 1
    return p_dict
    
def count_points(points_list):
    points_counts = dict(zip(points_list, [points_list.count(x) for x in points_list]))
    return points_counts

def get_overlap_spots(points_dict):
    return [k for k, v in points_dict.items() if v > 1]

def run_vent_counter(data, diag=False):
    inp = get_pairs(data)
    if diag:
        valid_points = inp
        
    else:
        valid_points = remove_diags(inp)
    point_frq = get_all_points(valid_points)
    return len(get_overlap_spots(point_frq))

In [45]:
%%time
run_vent_counter(test, False)

CPU times: user 961 µs, sys: 262 µs, total: 1.22 ms
Wall time: 1.11 ms


5

In [44]:
%%time
run_vent_counter(data, False)

CPU times: user 127 ms, sys: 34.3 ms, total: 162 ms
Wall time: 258 ms


5280

In [42]:
%%time
run_vent_counter(test, True)

CPU times: user 730 µs, sys: 193 µs, total: 923 µs
Wall time: 789 µs


12

In [43]:
%%time
run_vent_counter(data, True)

CPU times: user 161 ms, sys: 9.78 ms, total: 170 ms
Wall time: 176 ms


16716